Source https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [26]:
import torch
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# 1. Load the data

In [27]:
input_path = "./fruits-360/"

In [28]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'Training':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'Test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'Training':
    datasets.ImageFolder(input_path + 'Training', data_transforms['Training']),
    'Test':
    datasets.ImageFolder(input_path + 'Test', data_transforms['Test'])
}

dataloaders = {
    'Training':
    torch.utils.data.DataLoader(image_datasets['Training'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'Test':
    torch.utils.data.DataLoader(image_datasets['Test'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  # for Kaggle
}

#2. Define a Convolutional Neural Network

In [29]:
import torch.nn as nn
import torch.nn.functional as F

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [31]:
model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
               nn.Linear(2048, 512),
               nn.ReLU(inplace=True),
               nn.Linear(512, 131)).to(device)

# 3. Define a Loss function and optimizer

In [32]:
import torch.optim as optim

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

# 4. Train the network

In [34]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['Training', 'Test']:
            if phase == 'Training':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'Training':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

# 5. Test the network on the test data

In [35]:
model_trained = train_model(model, criterion, optimizer, num_epochs=3)

Epoch 1/3
----------


KeyboardInterrupt: 

In [ ]:
validation_img_paths = ["Test/Avocado/4_100.jpg",
                        "Test/Banana/12_100.jpg",
                        "Test/Cocos/0_100.jpg"]
img_list = [Image.open(input_path + img_path) for img_path in validation_img_paths]

In [ ]:
validation_batch = torch.stack([data_transforms['Test'](img).to(device)
                                for img in img_list])

In [ ]:
pred_logits_tensor = model(validation_batch)
pred_logits_tensor

In [ ]:
pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()
pred_probs

In [ ]:
fig, axs = plt.subplots(1, len(img_list), figsize=(20, 5))
for i, img in enumerate(img_list):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}% Avocado, {:.0f}% Banana, {:.0f}% Cocos".format(100*pred_probs[i,14],
                                                            100*pred_probs[i,15], 100*pred_probs[i,33]))
    ax.imshow(img)

# 6. Training on GPU

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# net.to(device)
# inputs, labels = data[0].to(device), data[1].to(device)

# 7. Save and load the network

In [ ]:
torch.save(model.state_dict(), 'weights.h5')

In [ ]:
model.load_state_dict(torch.load('weights.h5'))